In [1]:
def init(context):
    context.parameter = [55, 20, 10, 60, 20]
    context.tar = context.parameter[4]
    context.goods = ['CZCE.FG801', 'SHFE.rb1801']
    context.ratio = 0.8
    subscribe(symbols=context.goods, frequency='60s', count=101)
def on_bar(context, bars):
    bar = bars[0]
    symbol = bar['symbol']
    recent_data = context.data(symbol=symbol, frequency='60s', count=101, fields='close,high,low')
    close = recent_data['close'].values[-1]
    atr = talib.ATR(recent_data['high'].values, recent_data['low'].values, recent_data['close'].values,
                    timeperiod=context.tar)[-1]
    context.don_open = context.parameter[0] + 1
    upper_band = talib.MAX(recent_data['close'].values[:-1], timeperiod=context.don_open)[-1]
    context.don_close = context.parameter[1] + 1
    lower_band = talib.MIN(recent_data['close'].values[:-1], timeperiod=context.don_close)[-1]
    percent = context.ratio / float(len(context.goods))
    position_long = context.account().position(symbol=symbol, side=PositionSide_Long)
    position_short = context.account().position(symbol=symbol, side=PositionSide_Short)
    if not position_long and not position_short:
        ma_short = talib.MA(recent_data['close'].values, timeperiod=(context.parameter[2] + 1))[-1]
        ma_long = talib.MA(recent_data['close'].values, timeperiod=(context.parameter[3] + 1))[-1]
        dif = ma_short - ma_long
        if close > upper_band and (dif > 0):
            order_target_percent(symbol=symbol, percent=percent, order_type=OrderType_Market,
                                 position_side=PositionSide_Long)
            print(symbol, '市價多單: ', percent)
        if close < lower_band and (dif < 0):
            order_target_percent(symbol=symbol, percent=percent, order_type=OrderType_Market,
                                 position_side=PositionSide_Short)
            print(symbol, '市價多倉: ', percent)
    elif position_long:
        if close < lower_band:
            order_close_all()
            print(symbol, '市價平倉')
        else:
            vwap = position_long['vwap']
            money = position_long['cost']
            band = vwap - np.array([200, 2, 1.5, 1, 0.5, -100]) * atr
            grid_percent = float(pd.cut([close], band, labels=[0, 0.25, 0.5, 0.75, 1])[0]) * percent
            target_percent = np.minimum(money / context.account().cash['nav'], grid_percent)
            if target_percent != 1.0:
                print(symbol, '市價平倉多單: ', target_percent)
                order_target_percent(symbol=symbol, percent=target_percent, order_type=OrderType_Market,
                                     position_side=PositionSide_Long)
    elif position_short:
        if close > upper_band:
            order_close_all()
            print(symbol, '市價平倉')
        else:
            vwap = position_short['vwap']
            money = position_short['cost']
            band = vwap + np.array([-100, 0.5, 1, 1.5, 2, 200]) * atr
            grid_percent = float(pd.cut([close], band, labels=[1, 0.75, 0.5, 0.25, 0])[0]) * percent
            target_percent = np.minimum(money / context.account().cash['nav'], grid_percent)
            if target_percent != 1.0:
                order_target_percent(symbol=symbol, percent=target_percent, order_type=OrderType_Market,
                                     position_side=PositionSide_Short)
                print(symbol, '市價空單比例: ', target_percent)
